In [ ]:
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'
%load_ext autoreload
%autoreload 2

In [ ]:
import logging
import math
from abc import abstractmethod
from collections.abc import Callable, Iterator, Mapping, Sequence
from dataclasses import KW_ONLY, dataclass
from datetime import timedelta as py_td
from itertools import chain, count
from typing import (
    Any,
    ClassVar,
    Final,
    Generic,
    Literal,
    Optional,
    Protocol,
    TypeVar,
    cast,
    overload,
    reveal_type,
    runtime_checkable,
)

import numpy as np
import pandas as pd
from numpy.lib.stride_tricks import sliding_window_view
from numpy.typing import NDArray
from pandas import DataFrame, Index, Series, Timedelta, Timestamp
from tsdm.random.samplers import SlidingWindowSampler
from tsdm.types.protocols import Lookup
from tsdm.types.time import DTVar, NumpyDTVar, NumpyTDVar, TDVar
from tsdm.types.variables import (
    any_co as T_co,
    any_var as T,
    key_other_var as K2,
    key_var as K,
)
from tsdm.utils.data.datasets import Dataset, IterableDataset, MapDataset
from tsdm.utils.strings import pprint_repr

In [ ]:
tds = Series(pd.to_timedelta(np.random.rand(200), "m"))
tmin = pd.Timestamp(0)
tmax = tmin + pd.Timedelta(2, "h")
T = pd.concat([Series([tmin]), tmin + tds.cumsum(), Series([tmax])])
T = T.reset_index(drop=True)

stride = "5m"
# mode = "points"
horizons = "15m"
shuffle = False

sampler = SlidingWindowSampler(
    T, stride=stride, horizons=horizons, mode="points", shuffle=shuffle
)
indices = list(sampler)
X = DataFrame(np.random.randn(len(T), 2), columns=["ch1", "ch2"], index=T)
assert len(indices) >= 0 and len(X) > 0  # TODO: implement test

In [ ]:
MODE = TypeVar("MODE", Literal["masks"], Literal["slices"], Literal["points"])
Modes = TypeVar("Modes", bound=Literal["masks", "slices", "points"])

In [ ]:
class Foo: ...

In [ ]:
self = Foo()

data_source = T
horizons: str | Sequence[str] | NumpyTDVar | Sequence[NumpyTDVar]
mode: MODE = ("masks",)  # type: ignore[assignment
shuffle: bool = False
stride: str | NumpyTDVar
tmax: Optional[str | NumpyDTVar] = None
tmin: Optional[str | NumpyDTVar] = None
self.data = np.asarray(data_source)
self.mode = mode
self.stride = Timedelta(stride) if isinstance(stride, str) else stride

match tmin:
    case None:
        self.tmin = self.data.iloc[0] if isinstance(self.data, Series) else self.data[0]
    case str() as time_str:
        self.tmin = Timestamp(time_str)
    case _:
        self.tmin = tmin

match tmax:
    case None:
        self.tmax = (
            self.data.iloc[-1] if isinstance(self.data, Series) else self.data[-1]
        )
    case str() as time_str:
        self.tmax = Timestamp(time_str)
    case _:
        self.tmax = tmax

# this gives us the correct zero, depending on the dtype
self.zero_td = cast(NumpyTDVar, self.tmin - self.tmin)  # type: ignore[redundant-cast]
assert self.stride > self.zero_td, "stride cannot be zero."

# convert horizons to timedelta
horizons = Timedelta(horizons) if isinstance(horizons, str) else horizons
if isinstance(horizons, Sequence):
    self.multi_horizon = True
    if isinstance(horizons[0], str | Timedelta | py_td):
        self.horizons = pd.to_timedelta(horizons)
        concat_horizons = self.horizons.insert(0, self.zero_td)  # type: ignore[union-attr]
    else:
        self.horizons = np.array(horizons)
        concat_horizons = np.concatenate(([self.zero_td], self.horizons))  # type: ignore[arg-type]

    self.cumulative_horizons = np.cumsum(concat_horizons)
    self.total_horizon = self.cumulative_horizons[-1]
else:
    self.multi_horizon = False
    self.horizons = horizons
    self.total_horizon = self.horizons
    self.cumulative_horizons = np.cumsum([self.zero_td, self.horizons])

self.start_values = self.tmin + self.cumulative_horizons  # type: ignore[assignment, call-overload, operator]

self.offset = self.tmin + self.total_horizon  # type: ignore[assignment, call-overload, operator]

# precompute the possible slices
grid = compute_grid(self.tmin, self.tmax, self.stride, offset=self.offset)
self.grid = grid[grid >= 0]  # type: ignore[assignment, operator]

In [ ]:
isinstance("123", Sequence)

In [ ]:
self.data[0]

In [ ]:
self.tmin

In [ ]:
self.zero_td

In [ ]:
np.cumsum([self.zero_td, self.horizons])

In [ ]:
self.cumulative_horizons

In [ ]:
np.cumsum(np.array([self.zero_td, self.horizons]))

In [ ]:
from pydantic import BaseModel, computed_field, dataclasses


@dataclasses.dataclass(slots=True)
class Rectangle:
    width: int
    length: int

    @computed_field
    # @property
    def area(self) -> int:
        return self.width * self.length


# print(Rectangle(width=3, length=2).model_dump())
# > {'width': 3, 'length': 2, 'area': 6}

import numpy as np

obj = Rectangle(width=np.float32(3.0), length=2)
obj.area
# obj.__slots__

In [ ]:
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'
%load_ext autoreload
%autoreload 2

In [ ]:
import ast

print(
    ast.dump(
        ast.parse("""

from collections.abc import Sequence, Union


def foo(x) -> int | float: ...
"""),
        indent=4,
    )
)

In [ ]:
import torch
from torch import Tensor

In [ ]:
import array

arr = array.array("i", [1, 2, 3])

arr = bytes("agagaga", encoding="utf8")

In [ ]:
import pandas as pd

In [ ]:
x = [torch.randn(2), torch.randn(3)]

match x:
    case Tensor(a), Tensor(b):
        print(a, b)

In [ ]:
import warnings
from pathlib import Path
from typing import Any, Optional, Protocol, cast, runtime_checkable

import torch
import torch.utils.cpp_extension
from torch import Tensor

# constants
# we use FP32 machine epsilon as default tolerance
ATOL = 1e-6  # 2**-23  # ~1.19e-7
RTOL = 1e-6  # 2**-23  # ~1.19e-7

In [ ]:
def spectral_norm_debug(
    A: Tensor,
    u0: Optional[Tensor] = None,
    v0: Optional[Tensor] = None,
    maxiter: Optional[int] = None,
    atol: float = ATOL,
    rtol: float = RTOL,
) -> Tensor:
    """Computes the spectral norm."""
    return _spectral_norm_debug(A, u0, v0, maxiter, atol, rtol)

In [ ]:
type.__signature__

In [ ]:
from collections.abc import Sequence
from math import prod, sqrt
from typing import Optional, Protocol, Union, runtime_checkable

import torch
import torch.linalg
from numpy.typing import NDArray
from scipy import stats
from torch import BoolTensor, Tensor, jit, nn
from torch.optim import SGD

import linodenet
from linodenet.constants import TRUE
from linodenet.parametrize import *
from linodenet.projections import functional as projections
from linodenet.testing import check_jit_serialization
from linodenet.types import Device, Dtype, Shape

In [ ]:
import torch
import torch.nn as nn
import torch.nn.utils.parametrize as P

In [ ]:
U = torch.randn(5, 5)
x = torch.randn(5)

In [ ]:
torch.einsum("ij, k -> ik", U, x)

In [ ]:
from collections.abc import Mapping, Sized


class Foo(Sized):
    def __iter__(self): ...

    def __len__(self): ...


class Bar(Foo, Mapping):
    def __getitem__(self, key): ...


hash(Foo())  # ✔
hash(Bar())  # ✘ TypeError: unhashable type: 'Bar'

In [ ]:
bool((torch.linalg.matrix_rank(torch.randn(7, 5, 5)) <= 6).all())

In [ ]:
Bar.__eq__

In [ ]:
Bar.mro()

In [ ]:
class RankOne(nn.Module):
    def forward(self, x, y):
        # Form a rank 1 matrix multiplying two vectors
        return x.unsqueeze(-1) @ y.unsqueeze(-2)

    def right_inverse(self, Z):
        # Project Z onto the rank 1 matrices
        U, S, Vh = torch.linalg.svd(Z, full_matrices=False)
        # Return rescaled singular vectors
        s0_sqrt = S[0].sqrt().unsqueeze(-1)
        return U[..., :, 0] * s0_sqrt, Vh[..., 0, :] * s0_sqrt


model = nn.Linear(4, 4)
print(hash(model))
print(dict(model.named_parameters()))
linear_rank_one = P.register_parametrization(model, "weight", RankOne())
print(hash(linear_rank_one))

print(torch.linalg.matrix_rank(linear_rank_one.weight).item())

In [ ]:
dict(linear_rank_one.named_parameters())

In [ ]:
B, N, M = 7, 3, 5
inputs = torch.randn(B, N)
targets = torch.randn(B, M)
model = nn.Linear(in_features=N, out_features=M, bias=False)

In [ ]:
# register_parametrization(model, "weight", UpperTriangular)
param = UpperTriangular(model.weight)
delattr(model, "weight")
model.register_buffer("weight", param.cached_parameter)
model.register_module("weight_parametrization", param)
model.register_parameter("weight_original", param.original_parameter)
dict(model.named_parameters())

In [ ]:
scripted = jit.script(model)
dict(scripted.named_parameters())

In [ ]:
loaded = check_jit_serialization(scripted)
loaded.weight_parametrization.update_parametrization()
optim = SGD(loaded.parameters(), lr=0.1)
dict(loaded.named_parameters())

In [ ]:
with torch.no_grad():
    original_loss = (loaded(inputs) - targets).norm()
    print(original_loss)

loaded.weight, loaded.weight_original, loaded.weight_parametrization

In [ ]:
loaded.zero_grad(set_to_none=True)
loss = (loaded(inputs) - targets).norm()
print(loss)
loss.backward()
optim.step()
loaded.weight_parametrization.update_parametrization()

In [ ]:
dict(loaded.named_parameters())

In [ ]:
loss < original_loss